# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846852527609                   -0.70    4.8         
  2   -7.852320803220       -2.26       -1.53    1.0   17.2ms
  3   -7.852614765971       -3.53       -2.56    1.5   18.4ms
  4   -7.852645995622       -4.51       -2.87    2.5   22.8ms
  5   -7.852646485584       -6.31       -3.14    1.0   17.1ms
  6   -7.852646679057       -6.71       -3.97    1.0   17.1ms
  7   -7.852646686291       -8.14       -5.21    1.2   18.4ms
  8   -7.852646686721       -9.37       -5.37    2.5   23.4ms
  9   -7.852646686729      -11.10       -5.93    1.0   17.2ms
 10   -7.852646686730      -12.27       -6.14    1.8   20.3ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846889245995                   -0.70           4.8         
  2   -7.852554106681       -2.25       -1.64   0.80    2.2    205ms
  3   -7.852638471741       -4.07       -2.76   0.80    1.0   16.2ms
  4   -7.852646430805       -5.10       -3.33   0.80    2.0   20.9ms
  5   -7.852646678626       -6.61       -4.18   0.80    2.0   19.9ms
  6   -7.852646686434       -8.11       -4.76   0.80    2.0   19.7ms
  7   -7.852646686716       -9.55       -5.66   0.80    1.5   18.4ms
  8   -7.852646686730      -10.87       -6.50   0.80    2.2   21.0ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.428773e+01     3.560382e+00
 * time: 0.04464101791381836
     1     1.330746e+00     1.859564e+00
 * time: 0.21596598625183105
     2    -1.993969e+00     2.062790e+00
 * time: 0.23376202583312988
     3    -3.839732e+00     1.626343e+00
 * time: 0.25940895080566406
     4    -5.356170e+00     1.293101e+00
 * time: 0.2856941223144531
     5    -6.828882e+00     9.804964e-01
 * time: 0.3115251064300537
     6    -7.478695e+00     4.293986e-01
 * time: 0.33736395835876465
     7    -7.682212e+00     1.660531e-01
 * time: 0.3552210330963135
     8    -7.753392e+00     1.361277e-01
 * time: 0.3730480670928955
     9    -7.797310e+00     8.085739e-02
 * time: 0.39098095893859863
    10    -7.822311e+00     7.311244e-02
 * time: 0.40884995460510254
    11    -7.839630e+00     5.705225e-02
 * time: 0.42667603492736816
    12    -7.847676e+00     3.783725e-02
 * time: 0.4445509910583496
    13    -7.851033e+00     3.138471e-02
 * time: 0.46

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846770874233                   -0.70    4.2         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645647136                   -1.64         
  2   -7.852646686730       -5.98       -3.70    1.48s
  3   -7.852646686730      -12.84       -7.05    153ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 7.002158443812267e-7
|ρ_newton - ρ_scfv| = 1.3454665855594433e-7
|ρ_newton - ρ_dm|   = 1.2499678276302845e-9
